In [1]:
import numpy as np
import cv2
import glob
import os
import matplotlib.pyplot as plt

#images = glob.glob('./gopro-calibration/GOPR*.jpg')
images = glob.glob('./calibration_wide/GOPR*.jpg')

nx = 8
ny = 6

points = np.zeros((6*8, 3), np.float32)
points[:,:2] = np.mgrid[0:8, 0:6].T.reshape(-1, 2)

obj_points = []
img_points = []

print("Processing all files...")
for filename in images:
    print('Loading image', filename)
    image = cv2.imread(filename)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    print('Finding chessboard')
    ret, corners = cv2.findChessboardCorners(image, (nx, ny), None)
    print('Result is in hand', ret)
    
    if ret:
        obj_points.append(points)
        img_points.append(corners)
        
        #print('Found chessboard corners')
        #cv2.drawChessboardCorners(image, (8,6), corners, ret)
        #plt.imshow(image, cmap='gray')
        #plt.show()
    else:
        print('Chessboard cannot find corneres for:', filename)

print('All corners have been identified. You can use obj_points, img_points')
#print(all_locations)

Processing all files...
All corners have been identified. You can use obj_points, img_points


In [2]:
import pickle

%matplotlib inline

test_file = './calibration_wide/test_image.jpg'
print('Testing on: ', test_file)

img = cv2.imread(test_file)
img_size = img.shape[1], img.shape[0]

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, img_size,None,None)
if ret:
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    f, (i1, i2) = plt.subplots(1,2)
    i1.imshow(dst)
    i2.imshow(img)
    
filename = "wide_dist_pickle.p"
with open(filename, "wb") as f:
    pickle.dump({'mtx': mtx, 'dist': dist}, f)
    print("mtx and dist arrays have been saved to:", filename)

Testing on:  ./calibration_wide/test_image.jpg


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
import cv2

image = cv2.imread('traffic_sample.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

plt.plot(130, 62, '.')
plt.plot(200, 40, '.')
plt.plot(210, 140, '.')
plt.plot(122, 156, '.')

#pts = (130,62, 200, 40, 210, 140, 122, 156)

plt.show()

src = np.float32([
    [130,62],
    [122,156],
    [210,140],
    [200,40]
])

dst = np.float32([
    [130, 62],
    [130, 156],
    [227, 156],
    [227, 62]
])

# dst = np.float32([
#     [50, 50],
#     [50, 200],
#     [230, 200],
#     [230, 50]
# ])

print(src.shape)
print(dst.shape)

img_size = image.shape[1], image.shape[0]
M = cv2.getPerspectiveTransform(src, dst)
warped = cv2.warpPerspective(image, M, img_size, flags=cv2.INTER_LINEAR)

plt.imshow(warped)
plt.show()

In [ ]:
import numpy as np
import cv2
import glob
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

%matplotlib inline

filename = './calibration_wide\\GOPR0032.jpg'
image = mpimg.imread(filename)
image = cv2.undistort(image, mtx, dist, None, mtx)

ret, corners = cv2.findChessboardCorners(image, (nx, ny), None)
corners = corners.reshape(-1, corners.shape[2])

src_pts = np.float32([
    corners[0],
    corners[(ny - 1) * nx],
    corners[(ny * nx) -1],
    corners[nx - 1]
    ])

dest_pts = np.float32([
    corners[0],
    [corners[0][0], corners[(ny - 1) * nx][1]],
    [corners[nx - 1][0], corners[(ny - 1) * nx][1]],
    [corners[nx - 1][0], corners[0][1]],
])

print(src_pts)
print(dest_pts)

def draw_rectangle(pts):
    f, (ax1) = plt.subplots(1,1)
    ax1.add_patch(patches.Polygon(pts, closed=False))
    ax1.imshow(image)

draw_rectangle(src_pts)
draw_rectangle(dest_pts)
M = cv2.getPerspectiveTransform(src_pts, dest_pts)
img_size = image.shape[1], image.shape[0]
warped = cv2.warpPerspective(image, M, img_size, flags=cv2.INTER_LINEAR)    

f, (ax1) = plt.subplots(1,1)
ax1.imshow(warped)


In [ ]:
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

%matplotlib inline
# Read in the saved camera matrix and distortion coefficients
# These are the arrays you calculated using cv2.calibrateCamera()
dist_pickle = pickle.load( open( "wide_dist_pickle.p", "rb" ) )
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

# Read in an image
#filename = 'test_image2.png'
filename = './calibration_wide\\GOPR0032.jpg'
img = cv2.imread(filename)
nx = 8 # the number of inside corners in x
ny = 6 # the number of inside corners in y

# MODIFY THIS FUNCTION TO GENERATE OUTPUT 
# THAT LOOKS LIKE THE IMAGE ABOVE
def corners_unwarp(img, nx, ny, mtx, dist):
    # Pass in your image into this function
    # Write code to do the following steps
    undistort = cv2.undistort(img, mtx, dist, None, mtx)
    gray = cv2.cvtColor(undistort, cv2.COLOR_RGB2GRAY)
    
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    if ret:
        cv2.drawChessboardCorners(undistort, (nx, ny), corners, ret)

        corners = corners.reshape(-1, 2)
        
        img_size = undistort.shape[1], undistort.shape[0]
        margin = 50
        
        src = np.float32([
            corners[0],
            corners[(ny - 1) * nx],
            corners[-1],
            corners[nx - 1]
            ])
        
        dst = np.float32([
            [margin, margin], 
            [margin, img_size[1] - margin],
            [img_size[0] - margin, img_size[1] - margin],
            [img_size[0] - margin, margin]
        ])

        def draw_rectangle(pts):
            f, (ax1) = plt.subplots(1,1)
            ax1.add_patch(patches.Polygon(pts, closed=False))
            ax1.imshow(undistort)

        draw_rectangle(src)
        draw_rectangle(dst)
  
        M = cv2.getPerspectiveTransform(src, dst)
        warped = cv2.warpPerspective(undistort, M, img_size, flags=cv2.INTER_LINEAR)    

        return warped, M
    else:
        print('Ops.....Issue with chessboard')

top_down, perspective_M = corners_unwarp(img, nx, ny, mtx, dist)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(top_down)
ax2.set_title('Undistorted and Warped Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
